<a href='https://colab.research.google.com/github/rabbitmetrics/langchain-13-min/blob/main/notebooks/langchain-13-min.ipynb' target='_parent'><img src='https://colab.research.google.com/assets/colab-badge.svg' alt='Open In Colab'/></a>

### Read in pinecone vector database

In [1]:
# Load environment variables
import os
from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv())


True

In [2]:
# print(os.getenv('OPENAI_API_KEY'))
# print(os.getenv('PINECONE_ENVIRONMENT'))
# print(os.getenv('PINECONE_API_KEY'))

In [3]:
# Read existing vector index from pinecone
import pinecone
from langchain.vectorstores import Pinecone
from langchain.embeddings import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings(model='text-embedding-ada-002')
index_name = 'langchain-quickstart'
vectorstore = Pinecone.from_existing_index(index_name,embeddings_model)

c:\Python311\Lib\site-packages\pinecone\index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


### Query vector database

In [24]:
# query = 'What types of lubricants are to be avoided for mechanisms design?'
# query = 'What are examples of harmonic drive gearboxes for aerospace applications?'
# query = 'What types of deployable decelerators are there'
# query = 'What can you tell me about the Orion Side Hatch Design? Please explain any failures and lessons learned in detail'
query = 'What can you tell me about latch mechanism design failures which have occurred?'
# query = 'What can you tell me about ball-lock mechanism failures? Refer to specific examples.'

docs = vectorstore.similarity_search(query,k=6)
# docs_score = vectorstore.similarity_search_with_relevance_scores(query,k=4)

In [25]:
# Here's an example of the first document that was returned
print(docs[0].page_content[:450])

tree pointed to the latch release portion of the mechanism. High-speed video of the failure event showed 
the lever arm moving slightly during vibration with respect to the toggles, and then moving suddenly, and 
fully, in the direction of release. A cause for this behavior could not be found initially. Physical and 
dimensional inspection of the parts did not reveal any clear discrepancies. The engineering analysis 
77


### Query those docs to get an answer back

In [26]:
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain
from langchain import PromptTemplate

In [ ]:
llm = OpenAI(temperature=.5)
# chain = load_qa_chain(llm, chain_type="stuff", verbose=True)
# chain.run(input_documents=docs, question=query)

In [30]:
# Prompt
template = """You are an AI trained to be an expert in space mechanism design, analysis, test, and failure investigation.
Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer. 
Provide a summarized answer, followed up with examples in bullet points.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate(
    input_variables=["context", "question"],
    template=template,
)

# Chain
chain = load_qa_chain(llm, chain_type="stuff", prompt=QA_CHAIN_PROMPT, verbose=True)

# Run
resp=chain({"input_documents": docs, "question": query}, return_only_outputs=True)
print(resp['output_text'])




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are an AI trained to be an expert in space mechanism design, analysis, test, and failure investigation.
Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer. 
Provide a summarized answer, followed up with examples in bullet points.
tree pointed to the latch release portion of the mechanism. High-speed video of the failure event showed 
the lever arm moving slightly during vibration with respect to the toggles, and then moving suddenly, and 
fully, in the direction of release. A cause for this behavior could not be found initially. Physical and 
dimensional inspection of the parts did not reveal any clear discrepancies. The engineering analysis 
77

Launch Lock Mechanism Design Fault Tree Use and Coatings Study 
 
Daniel Villa* and Gustavo Toledo* 
 
 
Abstract 
Characte

### Stuff to try

In [9]:
# Add something with a system description so the chatbot knows some context.
# Try using the multi-context question tool as an input to a chain query.
# Use cosine scores to find more docs which are relevant.

# Try to create an agent to do a more extensive search.
#   Follow up from original prompt with a new question
#   Search for new docs based on the question
#   Plug new docs into another prompt with that question

In [ ]:
# TODO: #1 Check this out: https://python.langchain.com/docs/modules/data_connection/retrievers/ 
# TODO: #2 Check out how quivr uses langchain, copy key functionality

### Fancy stuff that is kinda sorta working

In [34]:
from langchain.chat_models import ChatOpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever
import logging

# logging.basicConfig()
# logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(), llm=llm, 
)
queries = retriever_from_llm.generate_queries(query)
unique_docs = retriever_from_llm.get_relevant_documents(query=query)

print(queries)
print(unique_docs)

TypeError: MultiQueryRetriever.generate_queries() got an unexpected keyword argument 'query'

In [11]:
# Generate
# https://python.langchain.com/docs/use_cases/question_answering/

# Return source docs
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=.5)
qa_chain = RetrievalQA.from_llm(llm,retriever=vectorstore.as_retriever(),
                                       return_source_documents=True)

results = qa_chain({'query': query})

In [12]:
print(results['query']+'\n')
print(results['result']+'\n')

print('Sources:')
for doc in results['source_documents']:
    print(doc.metadata)

What can you tell me about latch mechanism design? Please explain any failures and lessons learned in detail with specific examples.

The latch mechanism design for the Orion Side Hatch underwent testing and development, with several lessons learned and failures encountered. 

One lesson learned was the importance of considering spring hysteresis in the design. Spring hysteresis refers to the loss of spring force after cycling. The development testing confirmed that the spring force dropped by 10% - 20% after cycling. This hysteresis loss was compensated for in Moog's new latch system, which accounted for all environmental conditions and provided repeatable performance. The lesson here is that initial design development should include the consideration of spring force hysteresis losses.

Another lesson learned was the need for accurate modeling and analysis of complex latch mechanisms with many parts. The initial analysis of the latch design was limited to standard FEM/stress modeling 

In [13]:
# Lots of different ways to do this under https://python.langchain.com/docs/use_cases/question_answering/
from langchain.chains import RetrievalQAWithSourcesChain

qa_chain = RetrievalQAWithSourcesChain.from_llm(llm,retriever=vectorstore.as_retriever())

result = qa_chain({"question": query})
result

{'question': 'What can you tell me about latch mechanism design? Please explain any failures and lessons learned in detail with specific examples.',
 'answer': 'There is no specific information provided about latch mechanism failures or lessons learned in the given text.',
 'sources': ''}